In [ ]:
from web3 import Web3

RPC_URL = "https://eth-mainnet.g.alchemy.com/v2/G09aLwdbZ-zyer6rwNMGu"
w3 = Web3(Web3.HTTPProvider(RPC_URL))

print("Connected:", w3.is_connected())


In [ ]:
# Single runnable cell — copy/paste into Jupyter and run
from web3 import Web3
import json
import os
from datetime import datetime
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

# --- Connect to RPC ---
w3 = Web3(Web3.HTTPProvider("https://eth-mainnet.g.alchemy.com/v2/G09aLwdbZ-zyer6rwNMGu"))
print("Connected:", w3.is_connected())

# --- Fetch a block and one transaction ---
block_number = 17000000
block = w3.eth.get_block(block_number, full_transactions=True)

tx = block["transactions"][0]
tx_hash = tx["hash"]
receipt = w3.eth.get_transaction_receipt(tx_hash)

# --- Robust serializer ---
def to_serializable(obj):
    # Handle HexBytes / bytes
    if isinstance(obj, (HexBytes, bytes)):
        return obj.hex()
    # web3 often returns AttributeDict which behaves like dict but isn't JSON serializable
    if isinstance(obj, AttributeDict):
        obj = dict(obj)  # convert to plain dict and continue
    # Plain dict
    if isinstance(obj, dict):
        return {k: to_serializable(v) for k, v in obj.items()}
    # Lists/tuples
    if isinstance(obj, (list, tuple)):
        return [to_serializable(i) for i in obj]
    # Basic JSON-friendly scalar types
    if isinstance(obj, (str, int, float, bool)) or obj is None:
        return obj
    # Fallback: convert to string (safe)
    return str(obj)

receipt_json = to_serializable(receipt)

# --- Extract block timestamp (use timezone-aware UTC) ---
timestamp = block["timestamp"]
date_str = datetime.fromtimestamp(timestamp, datetime.UTC).strftime("%Y-%m-%d")
print("Date:", date_str)

# --- Save into date folder ---
folder = f"receipts/{date_str}"
os.makedirs(folder, exist_ok=True)
filename = f"{folder}/{tx_hash.hex()}_receipt.json"

with open(filename, "w") as f:
    json.dump(receipt_json, f, indent=4)

print("Saved:", filename)


In [ ]:
# Save all receipts from a block, each into the block date folder
from web3 import Web3
import json
import os
from datetime import datetime, timezone
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

w3 = Web3(Web3.HTTPProvider("https://eth-mainnet.g.alchemy.com/v2/G09aLwdbZ-zyer6rwNMGu"))
print("Connected:", w3.is_connected())

block_number = 17000000
block = w3.eth.get_block(block_number, full_transactions=True)

def to_serializable(obj):
    if isinstance(obj, (HexBytes, bytes)):
        return obj.hex()
    if isinstance(obj, AttributeDict):
        obj = dict(obj)
    if isinstance(obj, dict):
        return {k: to_serializable(v) for k, v in obj.items()}
    if isinstance(obj, (list, tuple)):
        return [to_serializable(i) for i in obj]
    if isinstance(obj, (str, int, float, bool)) or obj is None:
        return obj
    return str(obj)

timestamp = block["timestamp"]
date_str = datetime.fromtimestamp(timestamp, timezone.utc).strftime("%Y-%m-%d")
folder = f"receipts/{date_str}"
os.makedirs(folder, exist_ok=True)

saved = 0
for tx in block["transactions"]:
    tx_hash = tx["hash"]
    try:
        receipt = w3.eth.get_transaction_receipt(tx_hash)
        receipt_json = to_serializable(receipt)
        filename = f"{folder}/{tx_hash.hex()}_receipt.json"
        with open(filename, "w") as f:
            json.dump(receipt_json, f, indent=4)
        saved += 1
    except Exception as e:
        # skip failures but print them
        print(f"Failed to save {tx_hash.hex()}: {e}")

print(f"Done — saved {saved} receipts to {folder}")


In [1]:
# Optimized code: Fetch and save all transaction receipts from the latest block
from web3 import Web3
import json
import os
from datetime import datetime, timezone
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

# Connect to RPC
RPC_URL = "https://eth-mainnet.g.alchemy.com/v2/G09aLwdbZ-zyer6rwNMGu"
w3 = Web3(Web3.HTTPProvider(RPC_URL))
print("Connected:", w3.is_connected())

# Robust serializer function
def to_serializable(obj):
    if isinstance(obj, (HexBytes, bytes)):
        return obj.hex()
    if isinstance(obj, AttributeDict):
        obj = dict(obj)
    if isinstance(obj, dict):
        return {k: to_serializable(v) for k, v in obj.items()}
    if isinstance(obj, (list, tuple)):
        return [to_serializable(i) for i in obj]
    if isinstance(obj, (str, int, float, bool)) or obj is None:
        return obj
    return str(obj)

# Function to save a single receipt
def save_receipt(tx_hash, folder):
    try:
        receipt = w3.eth.get_transaction_receipt(tx_hash)
        receipt_json = to_serializable(receipt)
        filename = f"{folder}/{tx_hash.hex()}_receipt.json"
        with open(filename, "w") as f:
            json.dump(receipt_json, f, indent=4)
        return True
    except Exception as e:
        print(f"Failed to save {tx_hash.hex()}: {e}")
        return False

# Fetch latest block
block_number = w3.eth.block_number
print(f"Latest block number: {block_number}")
block = w3.eth.get_block(block_number, full_transactions=True)

# Extract date and create folder
timestamp = block["timestamp"]
date_str = datetime.fromtimestamp(timestamp, timezone.utc).strftime("%Y-%m-%d")
folder = f"receipts/{date_str}"
os.makedirs(folder, exist_ok=True)

# Save all receipts
saved = 0
for tx in block["transactions"]:
    if save_receipt(tx["hash"], folder):
        saved += 1

print(f"Done — saved {saved} receipts to {folder}")

Connected: True
Latest block number: 24232789
Done — saved 322 receipts to receipts/2026-01-14
